# Clustering Neighbourhoods in Toronto. Peer-Graded Assignment : (Part 1, 2 & 3)

By Jeeten Patel

Part 1 - Converting Wikipedia html table into a DataFrame

In [31]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [32]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [33]:
headers=table.findAll('th')
for i, head in enumerate(headers): headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")

In [34]:
rows=table.findAll('tr')
rows=rows[1:len(rows)]

for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")
    
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 2, expand = True) 
df.drop(columns=[0],inplace=True)
df = df.drop(df[(df.Borough == "Not assigned")].index)
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

df.Neighbourhood.fillna(df.Borough, inplace=True)

df=df.drop_duplicates()

df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))
df.update(
    df.Borough.loc[
       lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

In [35]:
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes
df2.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park (Toronto)
5,M9A,Queen's Park (Toronto),Queen's Park (Toronto)
6,M1B,"Scarborough, Toronto","Malvern, Toronto, Rouge, Toronto"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [36]:
df2.shape

(103, 3)

Part-2 Use the Geocoder package or the csv file to create the dataframe with Latitude & Longitude.

In [37]:
import pandas as pd
import io
import requests
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [47]:
c.rename({'Postal Code':'Postcode'},axis=1,inplace=True)

In [48]:
main=pd.concat([df2,c],join='outer',axis=1)

In [49]:
main

,Postcode,Borough,Neighborhood,Postcode,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,Regent Park,M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,Queen's Park (Toronto),M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M8X,Etobicoke,"Montgomery Road, Old Mill North, The Kingsway",M9N,43.706876,-79.518188
99,M4Y,Downtown Toronto,Church and Wellesley,M9P,43.696319,-79.532242
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,M9R,43.688905,-79.554724
101,M8Y,Etobicoke,"Fairmont Royal York Hotel, Kingsway Park South...",M9V,43.739416,-79.588437


Part-3 Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

In [52]:
main1=main[main['Borough'].str.contains('Toronto',na=False)].reset_index(drop=True)
main1.head()

,Postcode,Borough,Neighborhood,Postcode,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,M1E,43.763573,-79.188711
1,M7A,Downtown Toronto,Queen's Park (Toronto),M1H,43.773136,-79.239476
2,M9A,Queen's Park (Toronto),Queen's Park (Toronto),M1J,43.744734,-79.239476
3,M1B,"Scarborough, Toronto","Malvern, Toronto, Rouge, Toronto",M1K,43.727929,-79.262029
4,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848


In [53]:
main1.shape

(60, 6)

In [79]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge/label/cf201901 geopy # uncomment this line if you haven't completed the Foursquare API lab

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


CondaValueError: invalid package specification: #



Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [93]:

toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = main['Latitude']
Y = main['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
main['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(main['Latitude'], main['Longitude'], main['Borough'], main['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.8).add_to(toronto_map)  

toronto_map

Use the Foursquare API to explore the neighborhoods

In [82]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


In [96]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(main1['Latitude'], main1['Longitude'], main1['Postcode'], main1['Borough'], main1['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [95]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 9 elements